# **Multiclass Classification**

In [18]:
import os
import sys
from collections import Counter

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from torch.utils.data import dataloader, random_split
from torchinfo import summary
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

torch.backends.cudnn.deterministic = True

In [19]:
print("Platform:", sys.platform)
print("Python version:", sys.version)
print("---")
print("matplotlib version:", matplotlib.__version__)
print("pandas version:", pd.__version__)
print("PIL version:", PIL.__version__)
print("torch version:", torch.__version__)
print("torchvision version:", torchvision.__version__)

Platform: win32
Python version: 3.12.1 (tags/v3.12.1:2305ca5, Dec  7 2023, 22:03:25) [MSC v.1937 64 bit (AMD64)]
---
matplotlib version: 3.8.3
pandas version: 2.2.0
PIL version: 10.2.0
torch version: 2.5.1+cpu
torchvision version: 0.20.1+cpu


In [20]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"Using {device} device.")

Using cpu device.


**Project Goal**: To build and train a convolutional neural network, and use it to get predictions for the animal classes.

**Objectives**

- Read in data with multiple classes.
- Normalize data to improve performance.
- Create a CNN that works well with images.
- Train that network to do multiclass classification.
- Reformat the network predictions to complete the [competition](https://www.drivendata.org/competitions/87/competition-image-classification-wildlife-conservation/). - The goal of the competition is to build a model that takes an image and classifies what animal is in it. 

**1. Reading Files**

- Converting any grayscale images to RGB format with a custom class.

In [21]:
class ConvertToRGB:
    def __call__(self, img):
        if img.mode != "RGB":
            img = img.convert("RGB")
        return img

- Using transformers to resize the images, and converting the images to a Tensor of pixel values:

In [22]:
transform = transforms.Compose(
    [
        ConvertToRGB(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]
)

- Import data:

In [23]:
data_dir = ("data_p1/data_multiclass")
train_dir = os.path.join(data_dir, "train")

print("Will read data from", train_dir)

Will read data from data_p1/data_multiclass\train


In [24]:
dataset = datasets.ImageFolder(root=train_dir, transform=transform)

In [26]:
print(dataset)

Dataset ImageFolder
    Number of datapoints: 6050
    Root location: data_p1/data_multiclass\train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )
